In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Define system parameters
m = 0.1  # Mass of pendulum (kg)
M = 1.0  # Mass of cart (kg)
L = 1.0  # Length of pendulum (m)
g = 9.81  # Gravitational acceleration (m/s²)

# Kalman Filter (Placeholder - EKF/UKF would be needed for a full non-linear system)
# State: [x, theta, x_dot, theta_dot]

# Measurement noise covariance matrix (example values, can be tuned)
R_kf = np.diag([
    0.01**2,  # x position noise (std dev 0.01 m)
    (0.1 * np.pi / 180)**2,  # theta angle noise (std dev 0.1 degrees)
    0.05**2,  # x_dot velocity noise (std dev 0.05 m/s)
    (0.5 * np.pi / 180)**2  # theta_dot angular velocity noise (std dev 0.5 deg/s)
])

# Process noise covariance matrix (example values, tune based on model uncertainty)
# Represents uncertainty in the system model itself
Q_kf = np.diag([
    (0.001)**2,  # x process noise
    (0.001 * np.pi / 180)**2,  # theta process noise
    (0.01)**2,  # x_dot process noise
    (0.01 * np.pi / 180)**2  # theta_dot process noise
]) * 0.1 # Scaling factor for overall process noise

def simulate_sensor_data(true_state):
    """Simulates noisy sensor readings from the true state."""
    noise = np.random.multivariate_normal(np.zeros(len(true_state)), R_kf)
    return true_state + noise

def kalman_filter_step(previous_estimated_state, previous_covariance, noisy_measurement, control_input_F, dt):
    """
    Placeholder for a single step of a Kalman Filter (e.g., EKF or UKF).
    A full implementation would involve predict and update steps using the system model
    linearized around the current estimate, and the control_input_F.
    
    For now, this placeholder simply returns the noisy measurement as the current estimate
    and keeps the covariance matrix unchanged. This means the "filter" isn't actually
    filtering or predicting based on the model yet.
    """
    # In a real EKF:
    # 1. Predict step:
    #    - x_hat_pred = f(previous_estimated_state, control_input_F, dt) (non-linear state transition)
    #    - A = Jacobian of f w.r.t state
    #    - P_pred = A @ previous_covariance @ A.T + Q_kf
    # 2. Update step:
    #    - H = Jacobian of h (observation model, often identity if all states measured)
    #    - K = P_pred @ H.T @ inv(H @ P_pred @ H.T + R_kf)
    #    - current_estimated_state = x_hat_pred + K @ (noisy_measurement - h(x_hat_pred))
    #    - current_covariance = (I - K @ H) @ P_pred
    
    # Simplified placeholder:
    current_estimated_state = noisy_measurement 
    current_covariance = previous_covariance # No update to covariance in this placeholder
    return current_estimated_state, current_covariance

# Controller (PD Controller to stabilize theta around 0 - upright position)
Kp_theta = -2.0  # Proportional gain for theta (tuned for more aggressive response)
Kd_theta = -5.0  # Derivative gain for theta_dot (tuned for damping)
# Ki_theta = 5.0 # Integral gain (optional, can help with steady-state error)
# integral_error_theta = 0.0 # For I-term

def pid_controller(estimated_state, dt): # dt might be needed for integral term
    """Calculates the force F using a PD controller for the pendulum angle."""
    global integral_error_theta # if using I-term
    _, theta_est, _, theta_dot_est = estimated_state
    
    theta_desired = 0.0  # Target angle (upright)
    theta_dot_desired = 0.0  # Target angular velocity
    
    error_theta = theta_desired - theta_est
    error_theta_dot = theta_dot_desired - theta_dot_est
    
    # For I-term (optional)
    # integral_error_theta += error_theta * dt
    # integral_term = Ki_theta * integral_error_theta
    
    F_calculated = Kp_theta * error_theta + Kd_theta * error_theta_dot # + integral_term
    
    # Limit the force to avoid extreme values
    F_max = 25.0 # Max force in N (increased slightly)
    F_calculated = np.clip(F_calculated, -F_max, F_max)
    
    return F_calculated

# State-space representation
def system_dynamics(t, state, F_applied): # F_applied is now an argument
    """
    State space representation of the inverted pendulum on a cart.
    state = [x, theta, x_dot, theta_dot]
    F_applied: Force applied to cart (N)
    Returns: array of derivatives [x_dot, theta_dot, x_ddot, theta_ddot]
    """
    x, theta, x_dot, theta_dot = state
    
    # Equations of motion (derived from Lagrangian mechanics)
    # Denominator: M_total_eff = M + m * sin(theta)^2
    # This form is M + m - m*cos(theta)^2 which is M + m*sin(theta)^2
    # This denominator is always positive if M > 0 or m > 0.
    
    common_term_m_L_sin_theta = m * L * np.sin(theta)
    common_term_m_g_cos_theta = m * g * np.cos(theta)

    # x_ddot calculation
    numerator_x_ddot = F_applied + common_term_m_L_sin_theta * theta_dot**2 - common_term_m_g_cos_theta * np.sin(theta) # Corrected: m*g*sin(theta)*cos(theta)
    denominator_x_ddot = M + m * (np.sin(theta)**2) # M + m - m*cos(theta)^2
    if abs(denominator_x_ddot) < 1e-6: # Avoid division by zero if denominator is too small
        x_ddot = 0 # Or handle appropriately, though with M=1, m=0.1 this is unlikely
    else:
        x_ddot = numerator_x_ddot / denominator_x_ddot
    
    # theta_ddot calculation
    # theta_ddot = (g*np.sin(theta) - x_ddot*np.cos(theta)) / L
    # Substituting x_ddot can be complex. The original form is fine.
    # Ensure L is not zero
    if abs(L) < 1e-6:
        theta_ddot = 0 # Avoid division by zero
    else:
        theta_ddot = (g * np.sin(theta) - x_ddot * np.cos(theta)) / L
        
    return np.array([x_dot, theta_dot, x_ddot, theta_ddot])

# Simulation parameters for manual loop
t_start = 0.0
t_end = 1.0  # Increased simulation time to see stabilization
dt = 0.02  # Time step for the discrete simulation (20ms)
num_steps = int((t_end - t_start) / dt)

# Initial conditions
initial_state_true = np.array([0.0, np.pi / 4.0, 0.0, 0.0])  # x, theta (start further from upright), x_dot, theta_dot

# Kalman Filter Initialization
# Start with an estimate that might have some error compared to true initial state
current_estimated_state = initial_state_true.copy() + np.array([0.0, 0.00, 0.0, 0.0]) 
P_kf = np.diag([0.01**2, (0.1*np.pi/180)**2, 0.01**2, (0.1*np.pi/180)**2]) # Initial estimate covariance

# Data storage
history_t = [t_start]
history_true_state = [initial_state_true.copy()]
history_estimated_state = [current_estimated_state.copy()]
history_F = [0.0] # Initial force

current_true_state = initial_state_true.copy()
# integral_error_theta = 0.0 # Reset for PID if used

# Main simulation loop
for i in range(num_steps):
    t_current = t_start + i * dt
    
    # 1. Controller: Calculate force F based on (potentially noisy) estimated state
    F_applied = pid_controller(current_estimated_state, dt)
    
    # 2. System Dynamics: Update true state using Euler integration
    #    (For better accuracy, RK4 integration could be used here)
    derivatives = system_dynamics(t_current, current_true_state, F_applied)
    current_true_state += derivatives * dt
    
    # Normalize theta to be within [-pi, pi] for consistency
    current_true_state[1] = (current_true_state[1] + np.pi) % (2 * np.pi) - np.pi

    # 3. Simulate Sensor Data: Get noisy measurements from the true state
    noisy_measurements = simulate_sensor_data(current_true_state)
    
    # 4. Kalman Filter: Estimate state from noisy measurements
    #    (Using the placeholder KF step for now)
    current_estimated_state, P_kf = kalman_filter_step(current_estimated_state, P_kf, noisy_measurements, F_applied, dt)
    
    # Store data for plotting/animation
    history_t.append(t_current + dt)
    history_true_state.append(current_true_state.copy())
    history_estimated_state.append(current_estimated_state.copy())
    history_F.append(F_applied)

# Convert history to a structure that the animation function can use
class ManualSolution:
    def __init__(self, t_history, y_true_history, y_est_history, f_history):
        self.t = np.array(t_history)
        self.y = np.array(y_true_history).T  # Animate using the true state
        self.y_est = np.array(y_est_history).T # Estimated state also available
        self.f_applied = np.array(f_history) # Applied force history

solution = ManualSolution(history_t, history_true_state, history_estimated_state, history_F)

# Visualization
def create_animation():
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(-L*2.5, L*2.5) # Adjusted X limits based on pendulum length
    ax.set_ylim(-L*1.5, L*1.5) # Adjusted Y limits
    ax.set_aspect('equal')
    ax.grid(True)
    
    cart_width, cart_height = 0.4, 0.2
    
    # Create animation objects
    cart = plt.Rectangle((0, -cart_height/2), cart_width, cart_height, fc='royalblue', ec='black')
    pendulum, = ax.plot([], [], 'k-', lw=2.5) # Thicker pendulum
    mass, = ax.plot([], [], 'o', color='crimson', ms=10) # Pendulum mass
    time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')
    force_text = ax.text(0.05, 0.90, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')


    def init():
        ax.add_patch(cart)
        pendulum.set_data([], [])
        mass.set_data([], [])
        time_text.set_text('')
        force_text.set_text('')
        return cart, pendulum, mass, time_text, force_text
    
    def animate(i):
        # Ensure index is within bounds
        if i >= solution.y.shape[1]:
            return cart, pendulum, mass, time_text, force_text

        x_cart = solution.y[0, i]
        theta_pendulum = solution.y[1, i]
        
        # Update cart position
        cart.set_xy([x_cart - cart_width/2, -cart_height/2])
        
        # Calculate pendulum endpoint (pivot is at cart center, y=0)
        x_pend_end = x_cart + L * np.sin(theta_pendulum)
        y_pend_end = L * np.cos(theta_pendulum) # Pendulum hangs downwards from y=0 if theta=pi, upwards if theta=0
        
        pendulum.set_data([x_cart, x_pend_end], [0, y_pend_end])
        mass.set_data([x_pend_end], [y_pend_end])
        time_text.set_text(f'Time: {solution.t[i]:.2f}s')
        force_text.set_text(f'Force: {solution.f_applied[i]:.2f}N')

        return cart, pendulum, mass, time_text, force_text
    
    # Create animation
    # interval is dt in milliseconds
    anim = FuncAnimation(fig, animate, frames=len(solution.t), 
                         init_func=init, blit=True, interval=int(dt*1000))
    
    plt.close(fig) # Prevent duplicate static plot display in notebook
    return anim

# Create and display animation
animation = create_animation()
HTML(animation.to_jshtml())